In [ ]:
import os

import numpy as np
import pandas as pd
import glob
import xml.etree.ElementTree as ET
import argparse
import logging
import datetime
from pathlib import Path
import seaborn
import matplotlib.pyplot as plt
from pickle import dump,load

import sklearn
from sklearn.model_selection import GroupShuffleSplit,GroupKFold
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.feature_selection import VarianceThreshold, SelectFromModel
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score


%load_ext autoreload
%autoreload 1
%aimport load_data
%aimport evaluate
import load_data
import evaluate

xml_file_loc=''
imed_path=""

df = load_data.load_data(xml_file_loc, imed_path)

In [ ]:

df["time_since_onset"]=(df["ExaminationDate"] - df["Date of onset"]).apply(lambda x: x.days if not pd.isnull(x) else np.nan)
df["Age"]=df.apply(lambda x: x["ExaminationDate"].year - x["PatientBirthDate"], axis=1)

In [ ]:
from sklearn.model_selection import GroupShuffleSplit,GroupKFold

random_state=2024

create traing and test data

In [ ]:
df=df.dropna(subset=["EDSS"])

X=df.drop(columns=["EDSS","ExaminationDate",'PatientID','PatientBirthDate','Date of onset','EthnicGroup'])
groups=df["PatientID"]
Y=df["EDSS"]

outer_split=GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=random_state)
cv=GroupKFold(n_splits=5)


In [ ]:
for train_idx, test_idx in outer_split.split(X,Y,groups=groups):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    Y_train, Y_test = Y.iloc[train_idx], Y.iloc[test_idx]
    groups_train, groups_test = groups.iloc[train_idx], groups.iloc[test_idx]
    break


In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

from sklearn.pipeline import Pipeline

numerical_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import VarianceThreshold

numeric_transormer = Pipeline(steps=[("imputer",SimpleImputer(strategy='mean')),
                                     ("VarianceThreshold",VarianceThreshold(0.0)),
                                     ("scaler",StandardScaler())]).set_output(transform='pandas')

categorical_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="most_frequent")), 
           ("onehot", OneHotEncoder(drop='first',sparse_output=False)),]).set_output(transform='pandas')

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transormer, numerical_features),
        ("cat", categorical_transformer, categorical_features),
    ]
).set_output(transform='pandas')

In [ ]:
var=preprocessor.fit(X_train)["num"]["VarianceThreshold"].variances_
preprocessor.fit(X_train)["num"]["VarianceThreshold"].feature_names_in_[var<0.1]

In [ ]:
from sklearn.feature_selection import VarianceThreshold

feature_selector = Pipeline(steps=[("VarianceThreshold",VarianceThreshold(0.1)),
                                   
                                   ]).set_output(transform='pandas')


# Begin Modeling:
